# JAMy_cest_pas_sorcier Project
### Alexandre Flachs, Marie Giot & Jeanne Szpirer

#### Participation to "Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines" competition

First we need to charge all the data (training set features, test set features and training set labels).

In [ ]:
# Training set features
training_set_features <- read.csv("training_set_features.csv", stringsAsFactors = T)
dim(training_set_features)

In [ ]:
# Test set features
test_set_features <- read.csv("test_set_features.csv", stringsAsFactors = T)
dim(test_set_features)

In [ ]:
# Training set labels
training_set_labels <- read.csv("training_set_labels.csv", stringsAsFactors = T)
dim(training_set_labels)

We can see that the training features set and the training labels set has the same amount of lines, this is a first good sign because it means that we have an "answer" for every training line.
Then we can check if there is any missing value.

In [ ]:
# First method
dim(na.omit(training_set_features))
dim(na.omit(test_set_features))
dim(na.omit(training_set_labels))

# Second method (when TRUE, there is at least one missing value in the column)
apply(is.na(training_set_features),2,any)
apply(is.na(test_set_features),2,any)
apply(is.na(training_set_labels),2,any)

There are missing values in the features sets(13 506 lines instead of 26 707 in training set and 13 513 lines instead of 26 708 in test set). We can replace them with mean values like we did in the practicals. We are first going to transform the factor values into numerical ones.

In [ ]:
# On peut aussi regarder si certaines colonnes n'ont vraiment quasi aucune valeur, dans ce cas, ça vaut pas vraiment la peine de garder
sapply(training_set_features, function(x) sum(is.na(x)))

Quand même 12 274 valeurs manquantes pour "health_insurance" mais ça peut fort impacter le fait que le patient ait reçu une dose de vaccin. 

### Je le fais d'abord avec le training_set_features mais il faudra appliquer pareil au test_set_features je pense

In [ ]:
head(training_set_features)

In [ ]:
sapply(training_set_features[1,],class)

In [ ]:
factor_variables<-which(sapply(training_set_features[1,],class)=="factor")
factor_variables
data_factor<-training_set_features[,factor_variables]
dim(data_factor)
data_preprocessed<-training_set_features[,-factor_variables]

We can choose several variables that should influence the output variables in our opinion and according to the scientific articles we have read.
### Pas hésiter à les mettre quand on en aura hihi
Mais selon moi, "marital_status", "rent_or_own", "education" et "employment_status" devraient pas trop influencer. Et "employment_industry" & "employment_occupation" sont des random short strings hyper nombreux donc pas convaincue que ça soit très utile.

In [ ]:
# We will need dummies package to transform string values with one-hot-encoding.
install.packages('dummies')
library(dummies)

In [ ]:
# We keep only some factor variables
variables_to_keep<-c("age_group","race","sex","income_poverty","hhs_geo_region","census_msa")
data_factor_onehot <- dummy.data.frame(data_factor[,variables_to_keep], sep="_")

In [ ]:
dim(data_factor_onehot)

In [ ]:
data_factor_onehot[1:2,]

In [ ]:
data_preprocessed_extended<-cbind(data_preprocessed,data_factor_onehot)
dim(data_preprocessed_extended)

On peut passer au traitement des missing values. Pour le moment, j'implémente la même chose que dans les TP (on remplace par la mean value) mais on peut 100% faire plus de recherches et remplacer par autre chose. J'ai vu que la median value était souvent utilisée.

In [ ]:
replace_na_with_mean_value<-function(vec) {
    mean_vec<-mean(vec,na.rm=T)
    vec[is.na(vec)]<-mean_vec
    vec
}

In [ ]:
data_preprocessed_extended<-data.frame(apply(data_preprocessed_extended,2,replace_na_with_mean_value))
summary(data_preprocessed_extended)

In [ ]:
dim(na.omit(data_preprocessed_extended))

A ce stade, on a un set de data avec seulement des variables numériques qui ont du sens d'être là (selon moi hihi) et plus de missing values donc il manque surtout feature engineering. On pourra ensuite faire feature selection avec une mesure de la corrélation entre input et output.